Setup of our environment

In [1]:
#import ROOT

#Color printing
from termcolor import colored

#General data operations library
import math
import numpy as np

#HEP specific tools
import scipy.constants as scipy_constants

#Plotting libraries
import matplotlib.pyplot as plt

#Increase plots font size
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (10, 7),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [2]:
G_F = scipy_constants.physical_constants["Fermi coupling constant"]
m_e = scipy_constants.physical_constants["electron mass energy equivalent in MeV"]
m_mu = scipy_constants.physical_constants["muon mass energy equivalent in MeV"]

print("Fermi constant: {} +- {} {}".format(G_F[0], G_F[2], G_F[1]))
print("muon mass: \t{} +- {} {}".format(m_mu[0], m_mu[2], m_mu[1]))
print("electron mass: \t{} +- {} {}".format(m_e[0], m_e[2], m_e[1]))

G_F = G_F[0]
m_e = m_e[0]
m_mu = m_mu[0]

Fermi constant: 1.1663787e-05 +- 6e-12 GeV^-2
muon mass: 	105.6583755 +- 2.3e-06 MeV
electron mass: 	0.51099895 +- 1.5e-10 MeV


## Our homework functions

In [3]:
minS = 50
maxS = 200

m_Z = 92.1876
gamma_Z = 2.4952

def BreitWigner(s, m, gamma):
    gamma_small = np.sqrt(m ** 2 * (m ** 2 + gamma ** 2))
    k = (2 * np.sqrt(2) * m * gamma * gamma_small) / (np.pi * np.sqrt(m ** 2 + gamma_small))
    
    distr = k / ((s ** 2 - m ** 2) ** 2 + m ** 2 * gamma ** 2)
    return distr

def generateCandidateEnergy(nPoints):
    gamma = gamma_Z
    s = (maxS-minS)*np.random.random(nPoints) + minS
    x = np.random.random_sample(nPoints)
    bw_values = BreitWigner(s, m_Z, gamma)  # it is equivalent to p
    bw_values = bw_values / np.max(bw_values)
    accepted = x < bw_values
    return s[accepted]/2.0

def generateCandidateMomenta(energy):
    pn = 3
    p = np.random.random_sample(pn)
    sump = np.sum(p)
    p /= sump
    p *= energy
    
    px = p[0]
    py = p[1]
    pz = p[2]
    
    p4_electron = np.column_stack([energy, px,py,pz])
    p4_positron = np.column_stack([energy, -px,-py,-pz])
    return  p4_electron, p4_positron

def generateEvents(nPoints):
    energy = generateCandidateEnergy(nPoints)
    data = generateCandidateMomenta(energy)
    return data

def invMass(p4):
    metric = np.array([1,-1,-1,-1])
    p4_square = p4*(metric*p4)
    m = np.sqrt(np.sum(p4_square, axis=1))
    return m

## I have done here multiplication of two arrays; each row is multiplied by different value; thanks to this now we can generate N rows (for each energies) consisting of 3 momentum values


In [32]:
c = np.random.random_sample([3, 3]) 

csum = np.sum(c, axis=1)
print("\nMatrix c : \n", c)
print(csum)

cdiv = np.asanyarray(1 / csum)
cdivt = np.vstack(cdiv)
print(cdivt)

result_ar = np.dot(c ,cdivt)
r = np.multiply(c, cdivt)
print(r)


Matrix c : 
 [[0.67836661 0.68633197 0.59236832]
 [0.64908562 0.29179749 0.1595226 ]
 [0.83337309 0.45650139 0.09059301]]
[1.9570669  1.10040571 1.3804675 ]
[[0.51096874]
 [0.90875573]
 [0.72439228]]
[[0.34662413 0.35069418 0.30268169]
 [0.58986028 0.26517264 0.14496708]
 [0.60368904 0.33068608 0.06562488]]


## Here are energies generated

In [ ]:
nPoints = 5

e_min = 50
e_max = 200

energies = generateCandidateEnergy(nPoints)
print("Number of accepted events:\t",energies.shape[0])
print("Accepted events fraction:\t",energies.shape[0]/nPoints)


## Here is working generating momenta


In [ ]:
b = generateCandidateMomenta(energies)
b

## First test plots

In [ ]:
%%time

nPoints = 1000000
p4_electron, p4_positron = generateEvents(nPoints)
m = invMass(p4_electron + p4_positron)
#print("Generated {} events for {} tries. \nEfficiency: {:3.2f}".format(m.shape[0], nPoints, float(m.shape[0])/nPoints))

#fig, axes = plt.subplots(2,2, figsize=(12, 12))
#axes[0,0].plot(m, BreitWigner(m, m_Z, gamma_Z), "o",label="Breit-wigner");
#axes[0,0].hist(..., label=r"$e^{+}e^{-}$ MC data");
#axes[0,0].set_xlabel(r"invariant mass [GeV/$c^{2}$]")
#axes[0,0].set_ylabel("probability density")
#axes[0,0].legend(loc="upper right");